In [8]:
using Random

# parameters
n_disease = 40   # number of diseases
n_test = 30      # number of findings
expected_number_of_diseases = 3  # typical number of diags
sparseness1 = 0.5  # connection density

# the model
prev = (rand(1, n_disease) .+ 1) / 2  # prevalences of diseases, random between 0.5 and 1
prev .= expected_number_of_diseases * prev / sum(prev)  # in expectation sumofdiags diseases present

connections = rand(n_test, n_disease) .< sparseness1  # selection of connections
sensorig = (1 .+ 4 .* rand(n_test, n_disease)) ./ 10 .* connections  # sensitivities random between 0.1 and 0.5


30×40 Matrix{Float64}:
 0.0       0.0       0.0       0.105747  …  0.0       0.459955  0.416604
 0.0       0.0       0.0       0.0          0.42998   0.195314  0.0
 0.0       0.0       0.328993  0.378207     0.426252  0.0       0.31878
 0.0       0.0       0.38083   0.0          0.0       0.0       0.160173
 0.367093  0.200616  0.436865  0.0          0.0       0.243231  0.261676
 0.0       0.0       0.0       0.492325  …  0.0       0.241484  0.281887
 0.0       0.368226  0.448839  0.0          0.129511  0.341726  0.0
 0.0       0.0       0.330748  0.0          0.298143  0.179461  0.0
 0.0       0.0       0.0       0.303429     0.341852  0.173589  0.43337
 0.0       0.0       0.178385  0.104761     0.162747  0.0       0.0
 ⋮                                       ⋱                      
 0.350561  0.0       0.0       0.454234     0.0       0.0       0.0
 0.0       0.434459  0.0       0.0          0.0       0.189598  0.105968
 0.0       0.378896  0.238349  0.0          0.226148  0.0      

In [191]:

#=

# data generation. Generate diseases according to the prevalences. Then
# test results (=findings) according to the sensitivities.
# accept if the number of diseases and the number of positive findings is OK,
# else try again.
flag = true
while flag
    actualdiags = findall(rand(1, n_disease) .< prev)  # generate diseases
    ndiags = length(actualdiags)

    testresults = any(rand(n_test, ndiags) .< (sensorig[:, actualdiags]), dims=2)  # generate test results

    postest = findall(testresults .== 1)
    negtest = findall(testresults .== 0)
    m = length(postest)
    println([ndiags, m])
    if (m > 5) && (m < 15) && (!isempty(actualdiags))
        # for now, only cases with at least 5 positive tests, but no more than
        # 14 positive tests, and at least one disease.
        flag = false
    end
end

# gather data (some data preparation for the Quickscore algorithm)
sens = sensorig[postest, :]
sensneg = sensorig[negtest, :]

pfmin = 1 .- sens
pfminneg = prod(1 .- sensneg, dims=1)  # be careful with the dimensions: scalar result prod(1 - sensneg) for singleton neg test is wrong.

# Here it goes into quickscore.
tic()
(posterior, pfpluspd, pfplus) = fun_quickscoreW(prev, pfmin, pfminneg)
timep = toq()

actualdiseases = zeros(size(prev))
actualdiseases[actualdiags] .= 1
println("prevalences, posteriors, actual diseases")
println([prev', posterior', actualdiseases'])
println("\nPositive findings: $m, time: $timep")
```

Please note that there may be some differences in indexing and broadcasting between MATLAB and Julia, so I've adjusted the code accordingly. Also, Julia uses 1-based indexing, so some array indices have been adjusted accordingly.
=#
